In [2]:
import pandas as pd
import talib
import numpy as np

In [3]:
df = pd.read_csv('all_dailies.csv')

In [4]:
df['value'] = df['close'] * df['volume']

In [5]:
latest = df[df['date'] == df['date'].max()].copy()
latest.sort_values('value', ascending=False, inplace=True)

In [6]:
order = latest['ticker'].tolist()

In [7]:
rankdict = dict(zip(order, list(range(len(order)))))

In [8]:
def ranking(ticker):
    if ticker not in rankdict:
        return None
    return int(rankdict[ticker])

In [9]:
df['rank'] = df['ticker'].apply(ranking)

In [10]:
top = df[df['rank'] < 1000].copy()

In [11]:
def getdistance(vector, n=20):
    ma = talib.MA(vector, timeperiod=n)
    stdev = talib.STDDEV(vector, timeperiod=n, nbdev=1)
    return (vector - ma) / stdev

In [12]:
def talib_wrap(group, func, colname, *args, **kwargs):
    values = group[colname].values
    result = func(values, *args, **kwargs)
    return result

In [24]:
def gethighestdistance(df, n=20):
    df['distance'] = df['close'].groupby(df['ticker']).transform(getdistance)
    last = df[df['date'] == df['date'].max()].copy()
    lowest = last.nsmallest(n, 'distance').loc[:,['ticker', 'distance']]
    highest = last.nlargest(n, 'distance').loc[:,['ticker', 'distance']]
    return lowest, highest

In [35]:
def printhighestdistance(df, n=20):
    lowest, highest = gethighestdistance(df, n)
    print(str(n) + ' Lowest stocks by distance:')
    print(lowest.to_string(index=False))
    print('\n' + str(n) + ' Highest stocks by distance:')
    print(highest.to_string(index=False))